# Azure AI Search: Relevance

## Setup API clients

In [8]:
import os

import azure.identity
import dotenv
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from openai import OpenAI

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = OpenAI(
    base_url=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com/openai/v1",
    api_key=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
AZURE_SEARCH_FULL_INDEX = os.getenv("AZURE_SEARCH_INDEX", "gptkbindex")
AZURE_SEARCH_FIELD_NAME_EMBEDDING = os.getenv("AZURE_SEARCH_FIELD_NAME_EMBEDDING", "embedding")

search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)

#### Vector search is not always optimal

For example, consider searches for exact strings.

In [9]:
search_query = "$45.00"
search_vector = get_embedding(search_query)
r = search_client.search(None, top=3, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields=AZURE_SEARCH_FIELD_NAME_EMBEDDING)])
for doc in r:
    found_content = "Not found"
    if search_query.lower() in doc['content'].lower():
        found_content = doc['content'][doc['content'].lower().find(search_query.lower()):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 0.58118	Matching text: Not found
Score: 0.57963	Matching text: Not found
Score: 0.57403	Matching text: Not found


Compare to a text search for same query:

In [10]:
r = search_client.search(search_query, top=3)
for doc in r:
    found_content = "Not found"
    if search_query.lower() in doc['content'].lower():
        found_content = doc['content'][doc['content'].lower().find(search_query.lower()):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 2.60556	Matching text: $45.00</td><td>$55.00</td></tr><tr><td>Employee +1</td><td>$65.00</td><td>$71.00</td></tr><tr><td>Employee +2 or more</td><td>$78.00</td><td>$89.00</td></tr></table></figure>


#### Hybrid retrieval

Uses RRF to combine vector and text results.

In [12]:
search_vector = get_embedding(search_query)
r = search_client.search(search_query, top=15, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=10, fields=AZURE_SEARCH_FIELD_NAME_EMBEDDING)])
for doc in r:
    found_content = "Not found"
    if search_query in doc['content']:
        found_content = doc['content'][doc['content'].find(search_query):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 0.03205	Matching text: $45.00</td><td>$55.00</td></tr><tr><td>Employee +1</td><td>$65.00</td><td>$71.00</td></tr><tr><td>Employee +2 or more</td><td>$78.00</td><td>$89.00</td></tr></table></figure>
Score: 0.01667	Matching text: Not found
Score: 0.01639	Matching text: Not found
Score: 0.01613	Matching text: Not found
Score: 0.01587	Matching text: Not found
Score: 0.01562	Matching text: Not found
Score: 0.01515	Matching text: Not found
Score: 0.01493	Matching text: Not found
Score: 0.01471	Matching text: Not found
Score: 0.01449	Matching text: Not found


#### Hybrid ranking is not always optimal

In [14]:
search_query = "learning about underwater activities"
search_vector = get_embedding(search_query)
r = search_client.search(search_query, top=5, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=10, fields=AZURE_SEARCH_FIELD_NAME_EMBEDDING)])
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\t\tContent: {content}")

Score: 0.03137		Content:   PerksPlus covers a wide range of fitness activities, including but not limited to:  · Gym memberships  · Personal training sessions  · Yoga and Pila
Score: 0.02697		Content:   With PerksPlus, employees can choose from a variety of fitness programs to suit their individual needs and preferences. Whether you're looking to im
Score: 0.02597		Content: # Overview  Introducing PerksPlus - the ultimate benefits program designed to support the health and wellness of employees. With PerksPlus, employees 
Score: 0.02379		Content: 2\. Ask questions. If the employee is unsure about any part of the plan, it is important to ask questions in order to make sure that the plan is suita
Score: 0.02284		Content:   Seeking Pre-Approval for Care  In some cases, you may need to seek pre-approval or authorization before receiving certain medical services. This is 


#### Hybrid + semantic reranking 🎉

In [16]:
search_query = "learning about underwater activities"
search_vector = get_embedding(search_query)
r = search_client.search(
        search_query,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields=AZURE_SEARCH_FIELD_NAME_EMBEDDING)],
        query_type="semantic",
        semantic_configuration_name="default")

for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tReranker: {doc['@search.reranker_score']:.5f}\tContent: {content}")

Score: 0.03137	Reranker: 2.37414	Content:   PerksPlus covers a wide range of fitness activities, including but not limited to:  · Gym memberships  · Personal training sessions  · Yoga and Pila
Score: 0.02597	Reranker: 1.33856	Content: # Overview  Introducing PerksPlus - the ultimate benefits program designed to support the health and wellness of employees. With PerksPlus, employees 
Score: 0.02697	Reranker: 1.30170	Content:   With PerksPlus, employees can choose from a variety of fitness programs to suit their individual needs and preferences. Whether you're looking to im
Score: 0.02132	Reranker: 0.96566	Content:  Northwind Health Plus covers physical therapy services that are medically necessary.   ### Occupational Therapy  Occupational therapy helps individua
Score: 0.01639	Reranker: 0.95006	Content:  It is designed to help individuals with disabilities learn the skills they need to live as independently as possible, including communication, proble
